In [ ]:
import lightgbm as lgb
import numpy as np
import pandas as pd
import numpy as np
import gc
import os
import optuna
import sqlite3
import ray
import matplotlib.pyplot as plt
from optuna.integration import LightGBMPruningCallback
from autogluon.tabular import TabularPredictor
from sklearn.metrics import mean_squared_error, mean_absolute_error

# Rutas de entrada y salida
input_path = './data/l_vm_completa_normalizada_fe.parquet'
output_train = './data/df_train.parquet'
output_val = './data/df_val.parquet'

# Periodos para división
periodo_train_max = 201908
periodos_val = [201909, 201910]

In [ ]:


# Leer datos completos (solo una vez) y filtrar por los periodos necesarios
print("Leyendo y filtrando datos...")
df = pd.read_parquet(input_path, engine='fastparquet')

# Filtrar en dos partes directamente sin copias innecesarias
df_train = df[df['PERIODO'] <= periodo_train_max]
df_val = df[df['PERIODO'].isin(periodos_val)]

# Liberar el DataFrame original
del df
gc.collect()

# Guardar a disco
print("Guardando conjuntos en disco...")
df_train.to_parquet(output_train, index=False)
df_val.to_parquet(output_val, index=False)

# Liberar memoria final
del df_train, df_val
gc.collect()

print("✅ Proceso completado.")
print(f" - df_train → {output_train}")
print(f" - df_val   → {output_val}")


In [ ]:
df_train = pd.read_parquet(output_train, engine='fastparquet')
df_val = pd.read_parquet(output_val, engine='fastparquet')


In [ ]:
# Eliminar la columna CLASE_ZCORE de df_train y df_val
df_train.drop(columns=['CLASE_ZSCORE'], inplace=True)
df_val.drop(columns=['CLASE_ZSCORE'], inplace=True)

In [ ]:
# Mostrar las columnas del DataFrame de entrenamiento que contengan CLASE
print("Columnas que contienen 'CLASE' en df_train:")
clase_columns = [col for col in df_train.columns if 'CLASE' in col]
print(clase_columns)
print("Columnas que contienen 'CLASE' en df_val:")
clase_columns = [col for col in df_val.columns if 'CLASE' in col]
print(clase_columns)

In [ ]:
import datetime
import gc
from autogluon.tabular import TabularPredictor

gc.collect()

# === Configuración general ===
label_column = 'CLASE_DELTA_ZSCORE'
output_path = f"AutogluonModels/model_fast_{datetime.datetime.now().strftime('%Y%m%d_%H%M')}"

# === Entrenamiento ===
predictor = TabularPredictor(
    label=label_column,
    problem_type='regression',
    eval_metric='mean_absolute_error',
    path=output_path
).fit(
    train_data=df_train,
    tuning_data=df_val,
    use_bag_holdout=True,
    presets='best_quality',    
    time_limit=14400,
    num_bag_folds=3,                        # Reduce cantidad de folds
    num_bag_sets=1,                         # Una sola pasada
    auto_stack=False,                       # Apagar stacking para acelerar
    ag_args_ensemble={'fold_fitting_strategy': 'parallel_local'},
    hyperparameters={
        'GBM': [
             {
                'ag_args': {'name_suffix': 'Linear Tree'},
                'linear_tree': True,               
                'early_stopping_rounds': 10,
            },
            {
                'ag_args': {'name_suffix': 'Extra Trees'},
                'extra_trees': True,               
                'early_stopping_rounds': 10,
            },
            {
                'ag_args': {'name_suffix': 'QuickDefault'},
                'num_leaves': 31,
                'feature_fraction': 0.8,
                'bagging_fraction': 0.8,                
                'early_stopping_rounds': 10,
            }
        ]
    },
    verbosity=2
)


In [ ]:
predictor.fit_extra(
    time_limit=3600,
    ag_args_ensemble={'fold_fitting_strategy': 'sequential_local', 'num_bag_folds': 1},
    hyperparameters={
        'NN_TORCH': [{
            'num_epochs': 8,
            'learning_rate': 0.01,
            'dropout_prob': 0.1,
            'weight_decay': 1e-5,
            'batch_size': 256,
            'hidden_size': 128,
            'ag_args': {'name_suffix': 'NN_LightFast'},
            'ag_args_fit': {'num_gpus': 1}
        }]
    },
    fit_weighted_ensemble=True,
    verbosity=2
)


In [ ]:
predictor.fit_extra(
    time_limit=1800,  
    ag_args_ensemble={'fold_fitting_strategy': 'parallel_local'},
    fit_weighted_ensemble=True,
    hyperparameters={
        'CAT': [{
            'iterations': 500,
            'depth': 6,
            'learning_rate': 0.05,
            'early_stopping_rounds': 20,
            'ag_args': {'name_suffix': 'CAT_Custom'}
}]
    },
    verbosity=2
)

In [ ]:
predictor.fit_extra(
    time_limit=1800,  
    ag_args_ensemble={'fold_fitting_strategy': 'parallel_local'},
    fit_weighted_ensemble=True,
    hyperparameters={
        'XGB': [{
            'n_estimators': 150,
            'early_stopping_rounds': 10,
            'learning_rate': 0.05,
            'booster': 'gbtree',
            'ag_args': {'name_suffix': 'XGB_Quick'}
        }]
    },
    verbosity=2
)

In [ ]:
#predictor = TabularPredictor.load("AutogluonModels/model_fast_20250629_2217")

In [ ]:
# Comparar MAE y MedAE en el leaderboard
lb = predictor.leaderboard(data = df_val, extra_metrics=['mean_absolute_error', 'median_absolute_error', 'r2'], silent=False)

In [ ]:
df_importance = predictor.feature_importance(data=df_val, model='CatBoostCAT_Custom_BAG_L1')


In [ ]:
variables_irrelevantes = df_importance[df_importance["p_value"] > 0.99].index.tolist()

# Paso 2: Eliminar esas variables de los datasets de entrenamiento y validación
df_train_limpio = df_train.drop(columns=variables_irrelevantes, errors='ignore')
df_val_limpio = df_val.drop(columns=variables_irrelevantes, errors='ignore')

# (Opcional) Mostrar resumen
print(f"Se eliminaron {len(variables_irrelevantes)} variables irrelevantes.")
print("Primeras 50 variables eliminadas:", variables_irrelevantes[:50])


In [ ]:
# Guardar a disco
output_train_limpio = './data/df_train_limpio.parquet'
output_val_limpio = './data/df_val_limpio.parquet'
print("Guardando conjuntos en disco...")
df_train_limpio.to_parquet(output_train_limpio, index=False)
df_val_limpio.to_parquet(output_val_limpio, index=False)

In [ ]:
del df_train, df_val
gc.collect()

In [2]:
output_train_limpio = './data/df_train_limpio.parquet'
output_val_limpio = './data/df_val_limpio.parquet'

df_train_limpio = pd.read_parquet(output_train_limpio, engine='fastparquet')
df_val_limpio = pd.read_parquet(output_val_limpio, engine='fastparquet')

In [ ]:
""" target_col = 'CLASE_DELTA_ZSCORE'  # ajustá si tu columna objetivo es otra

hyperparameters = {
    'NN_TORCH': [
        {'ag_args': {'name_suffix': 'LightFast'}}
    ]
}

predictor_nn_lightfast = TabularPredictor(
    label='CLASE_DELTA_ZSCORE',
    problem_type='regression',
    eval_metric='mean_absolute_error',
    verbosity=3,
    path='AutogluonModels/nn_lightfast_retrain'
).fit(
    train_data=df_train_limpio,
    tuning_data=df_val_limpio,
    time_limit=7200,
    # hyperparameters={
    #     'NN_TORCH': [{
    #         'num_epochs': 8,
    #         'learning_rate': 0.01,
    #         'dropout_prob': 0.1,
    #         'weight_decay': 1e-5,
    #         'batch_size': 256,
    #         'hidden_size': 128,
    #         'ag_args': {'name_suffix': 'NN_LightFast'},
    #         'ag_args_fit': {'num_gpus': 1}
    #     }]
    # },
    hyperparameters={
        'NN_TORCH': {}
    },
    hyperparameter_tune_kwargs='auto',
    presets='best_quality',
    use_bag_holdout=False
) """

In [ ]:
target_col = 'CLASE_DELTA_ZSCORE'  # Ajustá si es necesario

df_entrenamiento_total = pd.concat([df_train_limpio, df_val_limpio], axis=0)
del df_train_limpio, df_val_limpio
gc.collect()

predictor_nn_lightfast = TabularPredictor(
    label=target_col,
    problem_type='regression',
    eval_metric='mean_absolute_error',
    verbosity=3,
    path='AutogluonModels/nn_lightfast_retrain'
).fit(
    train_data=df_train_limpio,
    tuning_data=df_val_limpio,
    time_limit=7200,
    presets='best_quality',
    hyperparameters={
        'NN_TORCH': {
            'ag_args': {'name_suffix': 'NN_LightFast'},
            'ag_args_fit': {'num_gpus': 1}
        }
    },
    hyperparameter_tune_kwargs='auto',
    use_bag_holdout=True
)

Verbosity: 3 (Detailed Logging)
=================== System Info ===================
AutoGluon Version:  1.3.1
Python Version:     3.9.23
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #28~24.04.1-Ubuntu SMP PREEMPT_DYNAMIC Fri May 23 10:31:01 UTC 2
CPU Count:          28
GPU Count:          1
Memory Avail:       89.02 GB / 125.58 GB (70.9%)
Disk Space Avail:   173.76 GB / 543.17 GB (32.0%)
Presets specified: ['best_quality']
============ fit kwarg info ============
User Specified kwargs:
{'auto_stack': True,
 'hyperparameter_tune_kwargs': 'auto',
 'num_bag_sets': 1,
 'use_bag_holdout': True}
Full kwargs:
{'_feature_generator_kwargs': None,
 '_save_bag_folds': None,
 'ag_args': None,
 'ag_args_ensemble': None,
 'ag_args_fit': None,
 'auto_stack': True,
 'calibrate': 'auto',
 'delay_bag_sets': False,
 'ds_args': {'clean_up_fits': True,
             'detection_time_frac': 0.25,
             'enable_callbacks': False,
             'enable_ray_logging': True,
     

KeyboardInterrupt: 

In [ ]:

# === 3. Evaluación del modelo ===

lb = predictor_nn_lightfast.leaderboard(df_val_limpio, extra_metrics=['mean_absolute_error', 'median_absolute_error', 'r2'], silent=False)

# === 4. Importancia de características ===

feature_importance = predictor_nn_lightfast.feature_importance(df_val_limpio)

# === 5. Guardado del modelo ya está hecho con `path=...` ===

print("✅ Reentrenamiento completado y modelo guardado en 'AutogluonModels/nn_lightfast_retrain'")


✅ Paso 1: Calcular errores de predicción

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Predicciones y error absoluto
df_val['y_true'] = df_val['CLASE_DELTA_ZSCORE']
df_val['y_pred'] = predictor.predict(df_val)
df_val['error_abs'] = abs(df_val['y_true'] - df_val['y_pred'])
df_val['error_signed'] = df_val['y_pred'] - df_val['y_true']


📊 Paso 2: Histogramas de error absoluto y error signado

In [ ]:
plt.figure(figsize=(10, 4))
sns.histplot(df_val['error_abs'], bins=50, kde=True)
plt.title('Distribución del Error Absoluto')
plt.xlabel('|y_pred - y_true|')
plt.ylabel('Frecuencia')
plt.show()

plt.figure(figsize=(10, 4))
sns.histplot(df_val['error_signed'], bins=50, kde=True)
plt.title('Distribución del Error Signado')
plt.xlabel('y_pred - y_true')
plt.ylabel('Frecuencia')
plt.axvline(0, color='red', linestyle='--')
plt.show()


📈 Paso 3: Error vs. Valor real (dispersión)


In [ ]:
plt.figure(figsize=(8, 6))
sns.scatterplot(x=df_val['y_true'], y=df_val['error_signed'], alpha=0.3)
plt.axhline(0, color='red', linestyle='--')
plt.title('Error Signado vs. Valor Real')
plt.xlabel('Valor real')
plt.ylabel('Error (pred - real)')
plt.show()


🧩 Paso 4: Promedio de error por grupo 

In [ ]:
top_errores_producto = df_val.groupby('PRODUCT_ID')['error_abs'].mean().sort_values(ascending=False).head(20)
top_errores_producto.plot(kind='bar', figsize=(10,4), title='Top 20 PRODUCT_ID con mayor error promedio')
plt.ylabel('Error Absoluto Medio')
plt.show()


In [ ]:
top_errores_producto = df_val.groupby('CUSTOMER_ID')['error_abs'].mean().sort_values(ascending=False).head(20)
top_errores_producto.plot(kind='bar', figsize=(10,4), title='Top 20 CUSTOMER_ID con mayor error promedio')
plt.ylabel('Error Absoluto Medio')
plt.show()

In [ ]:
df_full = pd.concat([df_train_limpio, df_val_limpio], ignore_index=True)
del df_train_limpio, df_val_limpio
gc.collect()
predictor_full = predictor_nn_lightfast.refit_full(train_data=df_full)
predictor_full.save("AutogluonModels/nn_lightfast_full")

In [ ]:
""" # Combinar entrenamiento + validación
df_full = pd.concat([df_train, df_val], axis=0)
del df_train, df_val
gc.collect() """

In [ ]:
# # Reentrenar el mejor modelo con TODOS los datos disponibles
# predictor_full = predictor.refit_full(train_data=df_full)

In [ ]:
# Verificar los modelos disponibles (el mejor ahora tiene el sufijo '_FULL')
print("Modelos disponibles luego del refit completo:")
print(predictor.leaderboard(silent=True)['model'].tolist())
# Eliminar modelos intermedios para liberar espacio
predictor.delete_models(models_to_keep='best', dry_run=False)

# Confirmar que solo queda el modelo reentrenado
print("\nModelos restantes después de eliminar los intermedios:")
print(predictor.leaderboard(silent=True)['model'].tolist())

# (Opcional) Guardar el predictor final si querés usarlo luego sin volver a cargar todo
predictor.save('./data/modelo_final_autogluon')

# ---  Liberar memoria ---
del df_full
gc.collect()


In [ ]:
# Cargo los datos sobre los que quiero hacer predicciones
df_pred_full = pd.read_parquet('./data/l_vm_completa_normalizada_fe.parquet', engine='fastparquet')
# Dejo solo los datos del periodo 201910 y que A_PREDECIR sea True
# Filtrar solo los datos del periodo 201910 y donde A_PREDECIR sea True
df_pred_full = df_pred_full[
    (df_pred_full['PERIODO'] == 201910) & (df_pred_full['A_PREDECIR'] == True)
].drop(columns=['CLASE_ZSCORE', 'CLASE_DELTA_ZSCORE'])

In [ ]:
# Realizar las predicciones usando el predictor original
predictions = predictor.predict(df_pred_full)
# Agregar las predicciones al DataFrame original
df_pred_full['CLASE_DELTA_ZSCORE'] = predictions

In [ ]:
# Imprimir la lista de columas del DataFrame con las predicciones
print("Columnas del DataFrame con las predicciones:")
print(df_pred_full.columns.tolist())

In [ ]:
# Dernormalizar la columna CLASE_DELTA_ZSCORE
df_pred_full['CLASE_DELTA'] = df_pred_full['CLASE_DELTA_ZSCORE'] * df_pred_full['CLASE_DELTA_STD'] + df_pred_full['CLASE_DELTA_MEAN']
df_pred_full['TN'] = df_pred_full['TN_ZSCORE'] * df_pred_full['TN_STD'] + df_pred_full['TN_MEAN']
# Agregar la columna TN_PREDICT que sea la suma de TN y CLASE_DELTA y si es menor que cero, poner cero
df_pred_full['TN_PREDICT'] = df_pred_full['TN'] + df_pred_full['CLASE_DELTA']
df_pred_full['TN_PREDICT'] = df_pred_full['TN_PREDICT'].clip(lower=0)

In [ ]:
# Generar Dataframe que contenga por cada PRODUCT_ID la suma de TN_PREDICT
df_final = df_pred_full.groupby('PRODUCT_ID').agg({'TN_PREDICT': 'sum'}).reset_index()
df_final = df_final.rename(columns={'PRODUCT_ID': 'product_id', 'TN_PREDICT': 'tn'})
# Guardar el DataFrame df_final en un archivo CSV
df_final.to_csv('./modelos/autoglun_normalizando_clase_delta.csv', index=False)
df_final.shape

# Para instalar AutoGluon con soporte para `autogluon.core.space` en conda, ejecuta:
# 
# conda install -c conda-forge autogluon
# 
# O si prefieres usar pip dentro de tu entorno conda:
# 
# pip install autogluon
# 
# Luego podrás usar:
# from autogluon.core import space as ag